## Database Creation

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
import re 
import requests
import json
from tqdm import tqdm
import datetime
import codecs
import unidecode
import contractions
from bs4 import BeautifulSoup 
import praw
import yfinance as yf
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from wordcloud import WordCloud
import nltk
from nltk.probability import FreqDist

### Data Collection

to create a dataset containing the WSB comments, we complete an existing dataset with more recent data

#### Kaggle  

the dataset can be downloaded [here](https://www.kaggle.com/theriley106/wallstreetbetscomments)

In [4]:
empty = []
for line in open('data/wsbData.json', 'r'):
    empty.append(json.loads(line))
df_kaggle = pd.DataFrame(empty)

format the dataframe : 

In [5]:
# convert unix timestamp
df_kaggle['date_created'] = pd.to_datetime(df_kaggle['created_utc'].astype(int), unit='s')
df_kaggle['date'] = df_kaggle['date_created'].dt.date
df_kaggle = df_kaggle.drop_duplicates(subset='body', keep="last")
df_kaggle = df_kaggle[["date", "body"]]
df_kaggle.columns = ["date", "text"]
df_kaggle = df_kaggle.set_index("date")

In [23]:
df_kaggle.to_pickle("data/df_kaggle.pkl")

In [6]:
len(df_kaggle)

2496853

#### Reddit API

In [7]:
# function to get data from pushshift api
def getPushshiftData(query, after, before, sub):
    url = 'https://api.pushshift.io/reddit/search/submission/?title='+str(query)+'&size=1000&after='+str(after)+'&before='+str(before)+'&subreddit='+str(sub)
    r = requests.get(url)
    data = json.loads(r.text)
    return data['data']

# get relevant data from data extracted using previous function
def collectSubData(subm):
    subData = [subm['id'], subm['title'], subm['url'], datetime.datetime.fromtimestamp(subm['created_utc']).date()]
    try:
        flair = subm['link_flair_text']
    except KeyError:
        flair = "NaN"
    subData.append(flair)
    subStats.append(subData)

request the data as of the last date of the kaggle dataset

In [8]:
# params
sub = 'wallstreetbets'
before = "1640217600" #dec 23 2021
after =  "1541030400" #nov 1 2018 
1541030400
query = "Daily Discussion Thread"
subCount = 0
subStats = []

data = getPushshiftData(query, after, before, sub)
while len(data) > 0:
    for submission in data:
        collectSubData(submission)
        subCount+=1
    after = data[-1]['created_utc']
    data = getPushshiftData(query, after, before, sub) 

In [9]:
# organize data into dataframe
data={}
ids=[]
titles=[]
urls=[]
dates=[]
flairs=[]
for stat in subStats:
    ids.append(stat[0])
    titles.append(stat[1])
    urls.append(stat[2])
    dates.append(stat[3])
    flairs.append(stat[4])
data['id']=ids
data['title']=titles
data['url']=urls
data['date']=dates
data['flair']=flairs

In [10]:
df_api = pd.DataFrame(data)

In [11]:
urls = dict(zip(df_api.id, df_api.url))

In [12]:
# connect to reddit api
reddit = praw.Reddit(client_id='OkOXFwArjdjJt3BBU9d_9A', client_secret='MnNW34sKNwhiLiOPN27YENdjVV5hdg', user_agent='wsb crawler',check_for_async=False)
comments_by_day = {}
# collect comments using praw
for id_, url in tqdm(urls.items()):
    try:
        submission = reddit.submission(url=url)
        submission.comments.replace_more(limit=0)
        comments= [(comment.body) for comment in submission.comments]
    except:
        comments=None
    comments_by_day[id_] = comments

100%|██████████| 798/798 [1:08:09<00:00,  5.12s/it]


In [13]:
df_api['comment'] = df_api['id'].map(comments_by_day)
df_api["comment"] = df_api["comment"].apply(lambda com: ". ".join(com) if com!=None else np.nan)
df_api = df_api[df_api['comment'].notna()]
df_api = df_api.drop_duplicates(subset='comment', keep="last")
df_api = df_api[["date", "comment"]]
df_api.columns = ["date", "text"]
df_api = df_api.set_index("date")

In [22]:
df_api.to_pickle("data/df_api.pkl")

In [14]:
len(df_api)

749

#### SPY prices

In [6]:
df_spy = yf.download('SPY', start='2011-01-01')
df_spy = df_spy.reset_index()

[*********************100%***********************]  1 of 1 completed


### Preprocess data

In [3]:
df_kaggle = pd.read_pickle("data/df_kaggle.pkl")
df_api = pd.read_pickle("data/df_api.pkl")

merge the dataframes and process the comments

In [4]:
df_base = df_kaggle.append(df_api)
df_base = df_base.dropna()
df_base = df_base.groupby("date")["text"].agg(lambda x: '. '.join(x)).to_frame()
df_base = df_base.reset_index()

In [6]:
df_base.date.min()

datetime.date(2012, 4, 11)

In [7]:
df_base.date.max()

datetime.date(2021, 12, 22)

add the spy prices and create labels

In [9]:
df_spy["label"] = df_spy["Close"].pct_change().shift(-1)
df_spy = df_spy.set_index("Date")
df_spy = df_spy.resample("D").asfreq().bfill()
# merge the SPY
df_base["date"] = pd.to_datetime(df_base["date"], format='%Y-%m-%d')
df_base = df_base.merge(df_spy, left_on='date', right_on='Date')
df_base = df_base.set_index('date')
df_base = df_base[df_base['label'].notna()]
df_base["label"] = np.where(df_base["label"]>0, 1, 0)
df_base = df_base[["text", "label"]]

In [11]:
df_base.to_csv("data/database_aug.csv")

In [10]:
len(df_base)

3019

### Data process

#### improve comments readability

In [ ]:
def remove_newlines_tabs(text): 
    # remove newline and tabs
    text = text.replace('\\n', ' ').replace('\n', ' ').replace('\t',' ').replace('\\', ' ').replace('. com', '.com')
    return text

def remove_deleted_username(text):
    # remove username "deleted"
    return text.replace('[deleted]', "")

def strip_html_tags(text):
    # remove html tags
    soup = BeautifulSoup(text, "html.parser")
    stripped_text = soup.get_text(separator=" ")
    return stripped_text

def remove_links(text):
    # remove links
    remove_https = re.sub(r'http\S+', '', text)
    remove_com = re.sub(r"\ [A-Za-z]*\.com", " ", remove_https)
    return remove_com

def remove_whitespace(text):
    # remove white spaces
    pattern = re.compile(r'\s+') 
    wo_whitespace = re.sub(pattern, ' ', text)
    text = wo_whitespace.replace('?', ' ? ').replace(')', ' ) ')
    return text

In [ ]:
df_base["text"] = df_base["text"].apply(remove_newlines_tabs)

In [ ]:
df_base["text"] = df_base["text"].apply(remove_deleted_username)

In [ ]:
df_base["text"] = df_base["text"].apply(strip_html_tags)

In [ ]:
df_base["text"] = df_base["text"].apply(remove_links)

In [ ]:
df_base["text"] = df_base["text"].apply(remove_whitespace)

#### cleaning the comments

In [ ]:
def accented_characters_removal(text):
    # remove accents
    try:
        decoded = unidecode.unidecode(codecs.decode(text, 'unicode_escape'))
    except:
        decoded = unidecode.unidecode(text)
    # format weird apostrophe
    apostrophe_handled = re.sub("’", "'", text)
    return apostrophe_handled

def lower_casing_text(text):
    return text.lower()

def reducing_incorrect_character_repeatation(text):
    # Pattern matching for all case alphabets
    pattern_alpha = re.compile(r"([A-Za-z])\1{1,}", re.DOTALL)
    # Limiting all the  repeatation to two characters.
    formatted_text = pattern_alpha.sub(r"\1\1", text) 
    # Pattern matching for all the punctuations that can occur
    pattern_punct = re.compile(r'([.,/#!$%^&*?;:{}=_`~()+-])\1{1,}')
    # Limiting punctuations in previously formatted string to only one.
    combined_formatted = pattern_punct.sub(r'\1', formatted_text)
    # The below statement is replacing repeatation of spaces that occur more than two times with that of one occurrence.
    final_formatted = re.sub(' {2,}',' ', combined_formatted)
    return final_formatted

def expand_contractions(text):
    # contractions expansion
    text = [contractions.fix(word) for word in text.split(' ')]
    text = ' '.join(text)
    return text

def removing_special_characters(text):
    # remove special characters
    text = re.sub(r"[^a-zA-Z0-9:$-,%.?!]+", ' ', text)
    return text

In [ ]:
df_base["text"] = df_base["text"].apply(accented_characters_removal)

In [ ]:
df_base["text"] = df_base["text"].apply(lower_casing_text)

In [ ]:
df_base["text"] = df_base["text"].apply(reducing_incorrect_character_repeatation)

In [ ]:
df_base["text"] = df_base["text"].apply(expand_contractions)

In [ ]:
df_base["text"] = df_base["text"].apply(removing_special_characters)

### Convert to CSV

In [ ]:
df_base.to_csv("data/database_aug_clean.csv")

### Wordcloud

Create a term document matrix on aggregated monthly discussion threads

In [ ]:
df_api = df_api.reset_index()
df_api.date = pd.to_datetime(df_api.date, format='%Y-%m-%d')
# corpus of aggreagated monthly discussion threads
df_wc = df_api.groupby([df_api.date.dt.year,df_api.date.dt.month])["text"].agg(lambda x: '. '.join(x))
df_wc = df_wc.to_frame()
corpus = df_wc.text.to_list()

In [ ]:
# term document matrix
vectorizer = TfidfVectorizer(stop_words='english', ngram_range = (1,1), max_df = .6, min_df = .01)
X = vectorizer.fit_transform(corpus)
feature_names = vectorizer.get_feature_names()
dense = X.todense()
denselist = dense.tolist()
df = pd.DataFrame(denselist, columns=feature_names)

In [ ]:
data = df.transpose()
data.columns = df_wc.index.to_list()
data.head()

Create the wordcloud with specific formatting

In [ ]:
def black_color_func(word, font_size, position,orientation,random_state=None, **kwargs):
    return("hsl(0,100%, 1%)")

In [ ]:
wordcloud = WordCloud(
    font_path = '/Library/Fonts/Arial Unicode.ttf', 
    background_color="white", 
    width=3000, 
    height=2000, 
    max_words=500
)

wordcloud.generate_from_frequencies(data[(2021, 1)]) # on January 2021
wordcloud.recolor(color_func = black_color_func)

In [ ]:
plt.figure(figsize=[15,10])
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.savefig('jan21_wc.png')